In [2]:
import glob
import csv
import pandas as pd
from pathlib import Path
from datetime import *


匯入sensor data

In [3]:
allFiles = glob.glob("../watch"+"*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    list_.append(file_)
list_ = sorted(list_)

csv_num = 7
for i in range(csv_num):
    if i==0:
        hR = pd.read_csv(list_[i])
        heading = pd.read_csv(list_[i+csv_num])
        xAcc = pd.read_csv(list_[i+2*csv_num])
        xMag = pd.read_csv(list_[i+3*csv_num])
        yAcc = pd.read_csv(list_[i+4*csv_num])
        yMag = pd.read_csv(list_[i+5*csv_num])
        zMag = pd.read_csv(list_[i+6*csv_num])
    else:
        hR = pd.concat([hR,pd.read_csv(list_[i])], join='inner').reset_index(drop=True)
        heading = pd.concat([heading, pd.read_csv(list_[i+csv_num])], join='inner').reset_index(drop=True)
        xAcc = pd.concat([xAcc,pd.read_csv(list_[i+2*csv_num])], join='inner').reset_index(drop=True)
        xMag = pd.concat([xMag,pd.read_csv(list_[i+3*csv_num])], join='inner').reset_index(drop=True)
        yAcc = pd.concat([yAcc,pd.read_csv(list_[i+4*csv_num])], join='inner').reset_index(drop=True)
        yMag = pd.concat([yMag,pd.read_csv(list_[i+5*csv_num])], join='inner').reset_index(drop=True)
        zMag = pd.concat([zMag,pd.read_csv(list_[i+6*csv_num])], join='inner').reset_index(drop=True)
    min_length = min(len(hR),len(heading),len(xAcc),len(xMag),len(yAcc),len(yMag), len(zMag))
    hR = hR[:min_length]
    heading = heading[:min_length]
    xAcc = heading[:min_length]
    xMag = xMag[:min_length]
    yAcc = yAcc[:min_length]
    yMag = yMag[:min_length]
    zMag = zMag[:min_length]


sensor data 與 activity 連結 
data_len筆資料算同一個data

In [36]:
sensors = [hR,heading,xAcc,xMag,yAcc,yMag,zMag]

# dt_obj = datetime.strptime(hR[0]['datetime'][0][2:], '%y-%m-%d %H:%M:%S')
# dt_ = datetime.strptime(hR[0]['datetime'][1][2:], '%y-%m-%d %H:%M:%S')
# print(abs(dt_-dt_obj))


data_len = 10

raw_act = pd.read_csv('../activity.csv')
act_index = 0
dt_act = datetime.strptime(raw_act['datetime'][0][2:], '%y/%m/%d %H:%M')
dt_act_next = datetime.strptime(raw_act['datetime'][1][2:], '%y/%m/%d %H:%M')
cur_act = raw_act['activity'][0]

data_list = []
data_count = 0

values = []
acts = []
val = []

for index in range(min_length):
    data_count = data_count + 1
    dt = datetime.strptime(hR['datetime'][index][2:], '%y-%m-%d %H:%M:%S')

    # val.append(dt)

    val.append(hR['value'][index])
    for s in sensors[1:]:
        val.append(s['value'][index])

    if(dt > dt_act_next and act_index < len(raw_act)-2):
        dt_act = dt_act_next
        act_index = act_index + 1
        dt_act_next = datetime.strptime(raw_act['datetime'][act_index+1][2:], '%y/%m/%d %H:%M')

    if(raw_act['activity'][act_index] != cur_act):
        cur_act = raw_act['activity'][act_index] 
        val = []
        data_count = 0

    if(data_count == data_len):
        acts.append(cur_act)
        values.append(val)
        val = []
        data_count = 0

    #

    # if((dt - dt_act).total_seconds() <= 20*60): # 20 mins
    #     val.append(raw_act['activity'][act_index])
    # else:
    #     val.append('NaN')

    # val.append(raw_act['activity'][act_index])

    # values.append(val)
    

In [38]:
print(values[0])


[106.0, 0.25088, 0.25088, -56.0, 809.0, -852.0, 221.0, 102.0, 0.256491, 0.256491, 91.0, 675.0, -632.0, 312.0, 97.0, 2.31249, 2.31249, -313.0, 754.0, -535.0, -490.0, 99.0, -1.20361, -1.20361, 643.0, 183.0, -82.0, -317.0, 104.0, -0.447343, -0.447343, 482.0, 718.0, -483.0, 285.0, 109.0, -0.6173, -0.6173, 365.0, 498.0, -560.0, -328.0, 113.0, -2.25246, -2.25246, 823.0, 767.0, -637.0, -158.0, 115.0, -2.73556, -2.73556, 791.0, 529.0, -687.0, -391.0, 110.0, 2.98672, 2.98672, 146.0, -316.0, -785.0, -158.0, 120.0, -2.48096, -2.48096, 261.0, 878.0, -340.0, -244.0]


In [42]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#shuffle data
temp = list(zip(values, acts))
shuffle(temp)
x, y = zip(*temp)

1962

In [13]:
# head = [['hR','heading','xAcc','xMag','yAcc','yMag','zMag','act']]
with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    # writer.writerows(head)
    writer.writerows(values)
